In [1]:
# 라이브러리 임포트
import os
import time
import json
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
#from fbprophet import Prophet
import pickle
import copy
from tqdm.auto import tqdm

In [2]:
from darts.models import *
from darts.models.forecasting import gradient_boosted_model
import darts.utils.timeseries_generation as tg
from darts.utils.likelihood_models import GaussianLikelihood
from darts.datasets import EnergyDataset
from darts.utils.missing_values import fill_missing_values
from darts import TimeSeries
from darts.dataprocessing.transformers import Scaler
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from darts.timeseries import TimeSeries
import warnings

warnings.filterwarnings("ignore")

[2022-06-14 13:40:05,718] WARNING | darts.models | Support for AutoARIMA, BATS and TBATS is not available.To enable it, install u8darts[pmdarima] or u8darts[all].
2022-06-14 13:40:05 darts.models WARNING: Support for AutoARIMA, BATS and TBATS is not available.To enable it, install u8darts[pmdarima] or u8darts[all].
[2022-06-14 13:40:05,720] WARNING | darts.models | Support for Facebook Prophet is not available. To enable it, install "darts", "u8darts[prophet]" or "u8darts[all]" (with pip);or "u8darts-all" (with conda).
2022-06-14 13:40:05 darts.models WARNING: Support for Facebook Prophet is not available. To enable it, install "darts", "u8darts[prophet]" or "u8darts[all]" (with pip);or "u8darts-all" (with conda).


In [1]:
import torch 
torch.cuda.device_count()

1

In [3]:
%%time
submit = json.load(open('./data/sample_submission/sample_submission.json', 'r', encoding='utf8')) 
df_dic = pickle.load(open('./mydata/df_dic.plk','rb'))
plc_lst = pickle.load(open('./mydata/plc_lst.plk','rb'))
fct_lst = ['pH', 'COD', 'SS', 'N', 'P', 'T']

CPU times: total: 719 ms
Wall time: 719 ms


In [4]:
# Prophet 예측을 위한 dataframe 포맷 제작
future = pd.DataFrame([str(x)[:10] for x in list(pd.date_range(start='2018-02-01', end='2019-12-31', inclusive="both"))], columns=['ds'])
future['y'] = np.nan
#future

---
# 예측 시작

In [16]:
dfall_comm= pickle.load( open('./mydata/dfall_comm.plk','rb'))
df_grp = dfall_comm.groupby('loc')
df_temp = df_grp.get_group(i)

---
## tcn

In [5]:
from darts.models.forecasting import tcn_model

In [ ]:
# 추론 : kalman
start_time = time.time()

for i in tqdm(plc_lst):
    plc_df = df_dic[i]
    
    for j in fct_lst: # 피쳐 리스트
        fct_df = plc_df[['ds', j]]   # 날짜 - 피쳐1 조합임.
        
        fct_df = fct_df.rename(columns={j:'y'}) # 피쳐1을 y로 바꾼다.
        fct_df['ds'] = fct_df.ds.apply(lambda x : datetime.strptime(x , '%Y%m%d'))
        fct_df = fct_df.dropna()
        fct_df['y'] = pd.to_numeric(fct_df.y)
        
        df_align = pd.DataFrame([ x for x in list(pd.date_range( end = '2018-01-31', periods = len(fct_df), inclusive="both"))], columns=['ds'])
        df_align['y'] = fct_df.y.values
        
        # TimeSeris 로 변환 하기 
        dfts = TimeSeries.from_dataframe(df_align,  time_col = 'ds' , value_cols  = ['y'], fill_missing_dates=True  , freq='D')
        
        diff = datetime(2018,2,1) - df_align.loc[0].ds
        if diff.days > 720:
            lag = 700
        else:
            lag = diff.days - 10
            
        my_model = tcn_model.TCNModel(
            input_chunk_length = 500,
            output_chunk_length = 1,
            n_epochs=50,
            kernel_size =3,
            num_filters = 12,
            dropout = 0.3,
            batch_size = 64,
            pl_trainer_kwargs  = {"accelerator": "gpu", "gpus": -1, "auto_select_gpus": True} ,
            dilation_base=2,
            weight_norm=True,
            random_state=2022,
            
            
        )
        
        my_model.fit(dfts)
        pred_series = my_model.predict( n = 699 )
        
        forecast = pred_series.pd_dataframe()
        forecast['ds'] = forecast.index
        forecast['ds'] = forecast['ds'].apply(lambda x : str(x).split(' ')[0].split('-')[0]+str(x).split(' ')[0].split('-')[1]+str(x).split(' ')[0].split('-')[2])
        forecast.columns = ['yhat', 'ds']
        forecast = forecast.reset_index(drop=True)
        
        for idx,k in enumerate(submit[i].keys()):
            submit[i][k][j] = np.round(forecast.yhat[idx],6)
      
        
end_time = time.time()
print('소요시간 :', (end_time - start_time)/60, 'minutes')

---

In [9]:
from darts.models.forecasting import nbeats

In [10]:
# 추론 : rf
start_time = time.time()

for i in tqdm(plc_lst):
    plc_df = df_dic[i]
    
    for j in fct_lst: # 피쳐 리스트
        fct_df = plc_df[['ds', j]]   # 날짜 - 피쳐1 조합임.
        
        fct_df = fct_df.rename(columns={j:'y'}) # 피쳐1을 y로 바꾼다.
        fct_df['ds'] = fct_df.ds.apply(lambda x : datetime.strptime(x , '%Y%m%d'))
        fct_df = fct_df.dropna()
        fct_df['y'] = pd.to_numeric(fct_df.y)
        
        df_align = pd.DataFrame([ x for x in list(pd.date_range( end = '2018-01-31', periods = len(fct_df), inclusive="both"))], columns=['ds'])
        df_align['y'] = fct_df.y.values
        
        # TimeSeris 로 변환 하기 
        dfts = TimeSeries.from_dataframe(df_align,  time_col = 'ds' , value_cols  = ['y'], fill_missing_dates=True  , freq='D')
        
        diff = datetime(2018,2,1) - df_align.loc[0].ds
        if diff.days > 720:
            lag = 700
        else:
            lag = diff.days - 10
            
        my_model = nbeats.NBEATSModel(
            input_chunk_length = 100,
            output_chunk_length = 1,
            generic_architecture=True,
            num_stacks=10,
            num_blocks=1,
            num_layers=4,
            layer_widths=512,
            n_epochs=2,
            nr_epochs_val_period=1,
            batch_size=256,
            model_name="nbeats_run"
        )
        
        my_model.fit(dfts)
        pred_series = my_model.predict( n = 699 )
        
        forecast = pred_series.pd_dataframe()
        forecast['ds'] = forecast.index
        forecast['ds'] = forecast['ds'].apply(lambda x : str(x).split(' ')[0].split('-')[0]+str(x).split(' ')[0].split('-')[1]+str(x).split(' ')[0].split('-')[2])
        forecast.columns = ['yhat', 'ds']
        forecast = forecast.reset_index(drop=True)
        
        for idx,k in enumerate(submit[i].keys()):
            submit[i][k][j] = np.round(forecast.yhat[idx],6)
      
        
end_time = time.time()
print('소요시간 :', (end_time - start_time)/60, 'minutes')

  0%|          | 0/545 [00:00<?, ?it/s]

[2022-06-14 14:53:50,584] INFO | darts.models.forecasting.torch_forecasting_model | Train dataset contains 2719 samples.
[2022-06-14 14:53:50,584] INFO | darts.models.forecasting.torch_forecasting_model | Train dataset contains 2719 samples.
2022-06-14 14:53:50 darts.models.forecasting.torch_forecasting_model INFO: Train dataset contains 2719 samples.
[2022-06-14 14:53:50,615] INFO | darts.models.forecasting.torch_forecasting_model | Time series values are 64-bits; casting model to float64.
[2022-06-14 14:53:50,615] INFO | darts.models.forecasting.torch_forecasting_model | Time series values are 64-bits; casting model to float64.
2022-06-14 14:53:50 darts.models.forecasting.torch_forecasting_model INFO: Time series values are 64-bits; casting model to float64.
2022-06-14 14:53:50 pytorch_lightning.utilities.rank_zero INFO: GPU available: False, used: False
2022-06-14 14:53:50 pytorch_lightning.utilities.rank_zero INFO: TPU available: False, using: 0 TPU cores
2022-06-14 14:53:50 pytorc

Training: 0it [00:00, ?it/s]

Predicting: 11it [00:00, ?it/s]

[2022-06-14 14:53:56,918] INFO | darts.models.forecasting.torch_forecasting_model | Train dataset contains 2719 samples.
[2022-06-14 14:53:56,918] INFO | darts.models.forecasting.torch_forecasting_model | Train dataset contains 2719 samples.
2022-06-14 14:53:56 darts.models.forecasting.torch_forecasting_model INFO: Train dataset contains 2719 samples.
[2022-06-14 14:53:56,949] INFO | darts.models.forecasting.torch_forecasting_model | Time series values are 64-bits; casting model to float64.
[2022-06-14 14:53:56,949] INFO | darts.models.forecasting.torch_forecasting_model | Time series values are 64-bits; casting model to float64.
2022-06-14 14:53:56 darts.models.forecasting.torch_forecasting_model INFO: Time series values are 64-bits; casting model to float64.
2022-06-14 14:53:56 pytorch_lightning.utilities.rank_zero INFO: GPU available: False, used: False
2022-06-14 14:53:56 pytorch_lightning.utilities.rank_zero INFO: TPU available: False, using: 0 TPU cores
2022-06-14 14:53:56 pytorc

...

Predicting: 11it [00:00, ?it/s]

[2022-06-14 15:27:45,580] INFO | darts.models.forecasting.torch_forecasting_model | Train dataset contains 2661 samples.
[2022-06-14 15:27:45,580] INFO | darts.models.forecasting.torch_forecasting_model | Train dataset contains 2661 samples.
2022-06-14 15:27:45 darts.models.forecasting.torch_forecasting_model INFO: Train dataset contains 2661 samples.
[2022-06-14 15:27:45,610] INFO | darts.models.forecasting.torch_forecasting_model | Time series values are 64-bits; casting model to float64.
[2022-06-14 15:27:45,610] INFO | darts.models.forecasting.torch_forecasting_model | Time series values are 64-bits; casting model to float64.
2022-06-14 15:27:45 darts.models.forecasting.torch_forecasting_model INFO: Time series values are 64-bits; casting model to float64.
2022-06-14 15:27:45 pytorch_lightning.utilities.rank_zero INFO: GPU available: False, used: False
2022-06-14 15:27:45 pytorch_lightning.utilities.rank_zero INFO: TPU available: False, using: 0 TPU cores
2022-06-14 15:27:45 pytorc

Training: 0it [00:00, ?it/s]

Predicting: 11it [00:00, ?it/s]

ValueError: __len__() should return >= 0